In [1]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint
import mlflow
import os

In [16]:
# Establecer la URI de la base de datos SQLite
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# Configuración de MLflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("arboles")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1713174288421, experiment_id='1', last_update_time=1713174288421, lifecycle_stage='active', name='arboles', tags={}>

# Sin Hiperparámetros

In [22]:
import preprocesamiento_datos
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import mlflow
import numpy as np

# Obtener los datos preprocesados
X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(True,[], False)

# Instanciar el clasificador de árbol de decisión con los parámetros predeterminados
clf = DecisionTreeRegressor(random_state=RANDOM_STATE)

# Entrenar el modelo
clf.fit(X_train, y_train)

# Predecir en el conjunto de validación
y_val_pred = clf.predict(X_val)

# Predecir en el conjunto de prueba
y_test_pred = clf.predict(X_test)

# Calcular el error absoluto medio en el conjunto de validación
val_mae = np.mean(np.abs(y_val - y_val_pred))

# Calcular el error absoluto medio en el conjunto de prueba
test_mae = np.mean(np.abs(y_test - y_test_pred))

# Impresión en pantalla de las predicciones en los conjuntos de validación y prueba
print("Predicciones en el conjunto de validación:")
print(val_mae)
print("Predicciones en el conjunto de prueba:")
print(test_mae)

# Iniciar el registro de la ejecución en MLflow
with mlflow.start_run(run_name="completo_default_param"):
    # Registrar el error absoluto medio en el conjunto de validación y prueba en MLflow
    mlflow.log_metric("val_mae", val_mae)
    mlflow.log_metric("test_mae", test_mae)


Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet
Predicciones en el conjunto de validación:
3664091.6808149405
Predicciones en el conjunto de prueba:
2949866.548042705


# Con Hiperparámetros

In [23]:
X_train, X_val, X_test, y_train, y_val, y_test, RANDOM_STATE = preprocesamiento_datos.preprocesamiento(True,[], False)
param_grid = {
    'max_depth': [5, 10, 30, 40, 50, 70, 100],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [40, 80, 90,100], 
    'min_samples_leaf': [5, 10, 20],
    'min_samples_split': [14, 30, 60], 
    'splitter': ['best', 'random']  
}

param_rand = {
    'splitter': ['best', 'random'],
    'max_depth': randint(3, 100),
    'min_samples_split': randint(2, 100),
    'min_samples_leaf': randint(1, 30),
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': randint(1, 200)
}

# Crear el estimador
clf = DecisionTreeRegressor(random_state=RANDOM_STATE)

# Estrategia de Grid Search
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=10, n_jobs=-1)

# Estrategia de Random Search
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_rand, n_iter=1000, scoring='neg_mean_absolute_error', cv=10, random_state=RANDOM_STATE, n_jobs=-1)

# Ejecutar las búsquedas
with mlflow.start_run(run_name="modelo_completo_gridSearch"):
    grid_search.fit(X_train, y_train)
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("grid_search_best_score", -grid_search.best_score_)
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model_grid_search")
    # Registrar el número de iteraciones
    mlflow.log_metric("grid_search_n_iter", len(grid_search.cv_results_['params']))

with mlflow.start_run(run_name="modelo_completo_randomizedSearch"):
    random_search.fit(X_train, y_train)
    mlflow.log_params(random_search.best_params_)
    mlflow.log_metric("random_search_best_score", -random_search.best_score_)
    mlflow.sklearn.log_model(random_search.best_estimator_, "model_random_search")
    # Registrar el número de iteraciones
    mlflow.log_metric("random_search_n_iter", len(random_search.cv_results_['params']))

Archivo Datos_la_liga_preparados_entrenamiento.parquet guardado en: ../Downloads\Datos_la_liga_preparados_entrenamiento.parquet


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Ar